In [23]:
import requests as requests
import json
import datetime
from datetime import date
import pandas as pd

keys={}
with open("/User/jgluck/Documents/GitHub/new-artists-twitter-bot/keys.json","r") as f:
    keys = json.loads(f.read())
    
# Consumer keys and access tokens, used for OAuth
songkick_api_key = keys['songkick_api_key']

In [142]:
def getArtistHistoryPage(artistId,page):
    
    url = "https://api.songkick.com/api/3.0/artists/" + str(artistId) + "/gigography.json"
    params = {'apikey': songkick_api_key, "page" : page, "per_page" : 50}
    resp = requests.get(url, params=params) 
    return json.loads(resp.text)


def wasArtistInCity(city, artistId):
    
    page = 1
    resp = getArtistHistoryPage(artistId, page)
    results = resp["resultsPage"]["results"]
    while (results != {}):
        events = results["event"]
        for event in events:
            if (city == event['location']['city']):
                return True

        page = page + 1
        resp = getArtistHistoryPage(artistId, page)
        results = resp["resultsPage"]["results"]
    
    return False

def getUpcomingShowsPage(metroId, minDate,MaxDate,page):
    
    url = "https://api.songkick.com/api/3.0/metro_areas/" + metroId + "/calendar.json"
    params = {'apikey': songkick_api_key, "min_date" : minDate, "max_date" : maxDate, "page" : page, "per_page" : 50}
    resp = requests.get(url, params=params) 
    return json.loads(resp.text)

def getUpcomingShows(daysAhead, metroId):
   
    artistIds = []
    artistNames = []
    artistUrls = []
    billings = []
    eventIds = []
    eventTypes = []
    eventUrls = []
    dates = []
    venueIds = []
    venueNames = []
    locationCities = []
    
    page = 1
    minDate = str(date.today())
    maxDate = str(date.today() + datetime.timedelta(daysAhead))
    resp = getUpcomingShowsPage(metroId, minDate,maxDate, page)
    results = resp["resultsPage"]["results"]
    
    while (results != {}):
        events = results["event"]
    
        for event in events:
            for performance in event["performance"]:
                artist = performance["artist"]
                if ("PRIVATE EVENT" not in artist["displayName"]):
                    billings.append(performance["billing"])
                    artistIds.append(artist["id"])
                    artistNames.append(artist["displayName"])
                    artistUrls.append(artist["uri"])
                    eventIds.append(event['id']) 
                    eventTypes.append(event['type'])
                    eventUrls.append(event['uri'])
                    dates.append(event['start']['date'])
                    venueIds.append(event['venue']['id'])
                    venueNames.append(event['venue']['displayName'])
                    locationCities.append(event['location']['city']) 
 
        page = page + 1
        resp = getUpcomingShowsPage(metroId, minDate,maxDate, page)
        results = resp["resultsPage"]["results"]
    
    data = pd.DataFrame(
            {
                "artistId" : artistIds,
                "artistName" : artistNames,
                "artistUrl" : artistUrls,
                "billing" : billings,
                "eventId" : eventIds,
                "eventType" : eventTypes,
                "eventUrl" : eventUrls,
                "date" : dates,
                "venueId" : venueIds,
                "venueName" : venueNames,
                "locationCity" : locationCities
            })
    
    return data

def runBot(days,cityName,cityId):
    
    artistsWhoPlayedInDC = []
    upcomingShows = getUpcomingShows(days,cityId)
    upcomingShows = upcomingShows[upcomingShows["locationCity"] == cityName]
    for artistId, artistName, artistUrl, venueName in zip(upcomingShows["artistId"],upcomingShows["artistName"],upcomingShows["artistUrl"],upcomingShows["venueName"]):

        if artistId not in artistsWhoPlayedInDC:
            if not wasArtistInCity(city, artistId):
                print("new artist: " + str(artistName) + " - " + str(artistId) + " is playing at " + venueName + " - " + artistUrl)
            artistsWhoPlayedInDC.append(artistId)      

In [ ]:
cityName = "Washington, DC, US"
cityId = "1409"
days = 7
runBot(days,cityName,cityId)

new artist: Black Midi - 9278794 is playing at U Street Music Hall - http://www.songkick.com/artists/9278794-black-midi?utm_source=59162&utm_medium=partner
new artist: Talulah Paisley - 10062890 is playing at Songbyrd Vinyl Lounge - http://www.songkick.com/artists/10062890-talulah-paisley?utm_source=59162&utm_medium=partner
new artist: Tim Atlas - 6990299 is playing at Union Stage - http://www.songkick.com/artists/6990299-tim-atlas?utm_source=59162&utm_medium=partner
new artist: NASAYA - 10017544 is playing at Union Stage - http://www.songkick.com/artists/10017544-nasaya?utm_source=59162&utm_medium=partner
new artist: bikini trill - 9187494 is playing at DC9 Nightclub - http://www.songkick.com/artists/9187494-bikini-trill?utm_source=59162&utm_medium=partner
new artist: Dylan Reese - 8743619 is playing at DC9 Nightclub - http://www.songkick.com/artists/8743619-dylan-reese?utm_source=59162&utm_medium=partner
new artist: Tokyo Machine - 9197194 is playing at Soundcheck - http://www.songki

new artist: The Kyle Sowashes - 890037 is playing at Slash Run - http://www.songkick.com/artists/890037-kyle-sowashes?utm_source=59162&utm_medium=partner
new artist: Dylan Mars - 9847419 is playing at Slash Run - http://www.songkick.com/artists/9847419-dylan-mars?utm_source=59162&utm_medium=partner
new artist: durandal - 9361224 is playing at sPACYcLOUd Lounge - http://www.songkick.com/artists/9361224-durandal?utm_source=59162&utm_medium=partner
new artist: Funkmod - 9570009 is playing at sPACYcLOUd Lounge - http://www.songkick.com/artists/9570009-funkmod?utm_source=59162&utm_medium=partner
new artist: Murkury - 9190649 is playing at sPACYcLOUd Lounge - http://www.songkick.com/artists/9190649-murkury?utm_source=59162&utm_medium=partner
new artist: Miss Money - 947726 is playing at Entertainment & Sports Arena - http://www.songkick.com/artists/947726-miss-money?utm_source=59162&utm_medium=partner
new artist: Dave - 13271 is playing at Union Stage - http://www.songkick.com/artists/13271-